In [1]:
import os
import pandas as pd
from collections import defaultdict

In [2]:
import sys
sys.path.append("D:/work/GitHub/air_pollution/")
#load the grouped list for colnames
from utils import *
# air_pollutions, weathers, weathers_2, times, optional_x

# 将每个站点的x和y的数据分别存在两个单独的csv里面

In [4]:
# illustrate the intersection
# of two lists in most simple way
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3


#write the x and y csv for each site
def write_xy_files(site_name, 
                   df_site, 
                   all_x_columns, 
                   all_y_columns, 
                   data_folder, 
                   output_folder):
    df = pd.read_csv(data_folder+site_name+'.csv')
    
    df['index'] = df.index
    #extract the included x features and y features
    df_x = df[['index']+intersection(['code']+all_x_columns, df.columns)]
    df_y = df[['index']+intersection(all_y_columns, df.columns)]
    #rename the code column for the later merge with site info
    df_x = df_x.rename(columns={"code": "ap_code"})
    
    #merge x dataframe with the site info
    df_x = df_x.merge(df_site, how='left', on=['ap_code'])
    
    df_x.to_csv(output_folder+site_name+'_x.csv', index=False)
    df_y.to_csv(output_folder+site_name+'_y.csv', index=False)

In [5]:
data_folder = "D:/work/污染项目/data-overall-update-0120/data/"
site_info_path = "D:/work/污染项目/data-overall-update-0120/siteinfo.csv"
output_folder = "D:/work/污染项目/data-overall-update-0120/data_process/before_engineer/"

In [6]:
#load the site info csv with info for each site
df_site = pd.read_csv(site_info_path)
#drop the Site and met columns, cause we already have their code column
df_site = df_site.drop(['Site', 'met'], axis=1)
df_site

,ap_code,ap_lat,ap_long,MetCode,met_lat,met_long,site
0,MY1,51.522530,-0.154611,37720,51.479167,-0.450556,UT
1,KC1,51.521050,-0.213492,37720,51.479167,-0.450556,UB
2,BIRR,52.476140,-1.874978,35340,52.453889,-1.748056,UT
3,BRS8,51.462839,-2.584482,37243,51.382778,-2.719167,UB
4,LED6,53.819970,-1.576361,33463,53.611667,-1.666944,UT
5,LEED,53.803780,-1.546472,33463,53.611667,-1.666944,UB
6,ABD7,57.144550,-2.106472,30910,57.205000,-2.205278,UT
7,AH,52.503850,-3.034178,35200,52.243056,-2.885833,RB
8,SCN2,53.586340,-0.636811,33735,53.306944,-0.548056,UI
9,YK11,53.951890,-1.075861,32660,54.043591,-1.250316,UT


In [9]:
#x, y data for each site
csv_files = os.listdir(data_folder)
csv_files = [item for item in csv_files if item.endswith('.csv')]

all_x_columns = weathers+weathers_2+times+optional_x
all_y_columns = air_pollutions

print ("all x columns: ")
print (all_x_columns)
print ("all y columns: ")
print (all_y_columns)

all x columns: 
['ws', 'wd', 'temp', 'RH', 'ssr', 'tp', 'blh', 'tcc', 'sp', 'tc', 'date_unix', 'week', 'weekday', 'hour', 'month', 'day_julian', 'o3']
all y columns: 
['nox', 'no2', 'no', 'o3', 'pm2.5', 'pm10']


In [10]:
#output the x and y files for each site
for site_name in csv_files:
    site_name = site_name[:-4]
    write_xy_files(site_name, 
                   df_site, 
                   all_x_columns, 
                   all_y_columns,
                   data_folder,
                   output_folder)

In [11]:
# last step, 检查是否除了tc以外，其他的columns每个站点都有？
x_files = os.listdir(output_folder)
x_files = [item for item in x_files if item.endswith('_x.csv')]


In [19]:
sites_name_with_o3 = []
for x_file in x_files:
    site_name = x_file[:-6]
    df_x = pd.read_csv(output_folder+x_file)
    colnames = df_x.columns
    
    for col in all_x_columns+['index']:
        if col not in colnames:
            print (col+" not in columns in "+site_name)

    if 'o3' in colnames:
        sites_name_with_o3.append(site_name)

tc not in columns in AberdeenUnionStreetRoadside
o3 not in columns in AberdeenUnionStreetRoadside
tc not in columns in AstonHill
tc not in columns in BirminghamA4540
tc not in columns in BristolStPauls
tc not in columns in LeedsCentre
tc not in columns in LeedsHeadingleyKerbside
o3 not in columns in LeedsHeadingleyKerbside
tc not in columns in NorthKensington
tc not in columns in ScunthorpeTown
o3 not in columns in ScunthorpeTown
tc not in columns in YorkFishergate
o3 not in columns in YorkFishergate


In [20]:
sites_name_with_o3

['AstonHill',
 'BirminghamA4540',
 'BristolStPauls',
 'LeedsCentre',
 'MaryleboneRoad',
 'NorthKensington']

# 将各个站点的数据union在一起，即一个y一个大的csv,包含所有站点的数据（先不考虑tc这个column）

In [21]:
def unoin_x_y_df(x_y_folder, 
                 site_name, 
                 selected_x_columns, 
                 air_pollution):
    #read x and y dataframes
    df_x = pd.read_csv(x_y_folder+site_name+'_x.csv')
    df_y = pd.read_csv(x_y_folder+site_name+'_y.csv')

    #selected columns in x and y dataframes
    df_x = df_x[selected_x_columns]
    df_y = df_y[['index']+[air_pollution]]

    #merge x dataframe with y use the index
    df = df_y.merge(df_x, how='left', on=['index'])

    return df


def union_df_perPollution(air_pollution, 
                          pollutions_include_sites, 
                          selected_x_columns,
                          x_y_folder,
                          union_output_path):
    included_sites = pollutions_include_sites[air_pollution]
    
    df_list = []
    for site_name in included_sites:
        # x csv 和 y csv 合并的时候用index去join
        df_list.append(unoin_x_y_df(x_y_folder, 
                                    site_name, 
                                    selected_x_columns, 
                                    air_pollution))
        
    df_union = pd.concat(df_list)
    #filter out NA ws column and NA y 
    #######################################################
    ### follow Grange and Carslaw原始论文里, 在训练模型前 ###
    ### 只提取了有wind speed 的所有数据进行训练 ##############
    #######################################################
    df_union = df_union[~pd.isnull(df_union['ws'])]
    df_union = df_union[~pd.isnull(df_union[air_pollution])]
    
    df_union.to_csv(union_output_path+air_pollution+'_unionSites.csv', index=False) 

In [39]:
union_output_path_noo3 = "D:/Work/污染项目/data-overall-update-0120/"\
                              "data_process/before_engineer/union_per_pollution/without_o3/"

union_output_path_o3 = "D:/Work/污染项目/data-overall-update-0120/"\
                              "data_process/before_engineer/union_per_pollution/with_o3/"

In [34]:
y_files = os.listdir(output_folder)
y_files = [item for item in y_files if item.endswith('_y.csv')]

In [35]:
#存储每个y(pollution)都包含哪一些站点(sites)
pollutions_include_sites = defaultdict(list)

for y_file in y_files:
    site_name = y_file[:-6]
    
    df_y = pd.read_csv(output_folder+y_file)
    y_colnames = df_y.columns
    
    for col in y_colnames:
        if col != 'index':
            pollutions_include_sites[col].append(site_name)
        

In [36]:
pollutions_include_sites.keys(), air_pollutions

(dict_keys(['nox', 'no2', 'no', 'o3', 'pm2.5', 'pm10']),
 ['nox', 'no2', 'no', 'o3', 'pm2.5', 'pm10'])

不加o3作为x,即全部站点一起union

In [37]:
#selected_x_columns for training
selected_x_columns = all_x_columns.copy()
selected_x_columns.remove('tc')
selected_x_columns.remove('o3')
selected_x_columns = ['index','ap_code']+selected_x_columns+['ap_lat', 'ap_long', 
                                                     'MetCode', 'met_lat', 
                                                     'met_long', 'site']

In [38]:
#save the unioned pollution csv
for air_pollution in pollutions_include_sites.keys():
    union_df_perPollution(air_pollution, 
                          pollutions_include_sites, 
                          selected_x_columns,
                          output_folder,
                          union_output_path_noo3)

加o3作为x,只有部分站点，即sites_name_with_o3一起union

In [40]:
#selected_x_columns for training
selected_x_columns = all_x_columns.copy()
selected_x_columns.remove('tc')
selected_x_columns = ['index','ap_code']+selected_x_columns+['ap_lat', 'ap_long', 
                                                     'MetCode', 'met_lat', 
                                                     'met_long', 'site']

In [50]:
pollutions_include_sites_o3 = {}

for air_pollution in pollutions_include_sites:
    if air_pollution != 'o3':
        value = []
        for site_name in pollutions_include_sites[air_pollution]:
            if site_name in sites_name_with_o3:
                value.append(site_name)
        pollutions_include_sites_o3[air_pollution] = value

In [53]:
#save the unioned pollution csv
for air_pollution in pollutions_include_sites_o3.keys():
    union_df_perPollution(air_pollution, 
                          pollutions_include_sites_o3, 
                          selected_x_columns,
                          output_folder,
                          union_output_path_o3)